In [ ]:
from dash import Dash, html, dcc, callback, Output, Input, State, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import base64
import io
from THz import THzSpec, THzSpecSet, THzData

# global for the data to process
data=None 


The system uses the concept of a THzSpectralSet which links the sample measurement with the reference measurement <br>
You can create a spectral data set  directly from the THZSpectralSet Class using <br>
``` 
data_set = THzSpecSet(sample_file, ref_file) 
```
The THz data class lets you store multiple THzSpectralSets in a pandas dataframe.  Additional kwargs provided with the add data command will be turned into columns in the dataframe

In [ ]:
reference_file = r"./ref2.dat"
sample_file = r"./samp2.dat"

#Create a THz sepctral Set
#THz_set = THzSpecSet(sample_file,reference_file)
# or just use the Spectral Set directly if created
#data = THz_set

# Or use the data tool

THz_data=THzData()
THz_data.add_data(sample_file,reference_file,'sample_1',time=26,conditions='cold')
THz_data.add_data(sample_file,reference_file,'sample_1',time=26,conditions='cold', temperature=293)
# ability to process all files with the same parameters
THz_data.process_all(0,30000,3,20)
#select a row of the THz_data frame
data = THz_data.df.iloc[0]['data']


In [ ]:
# Helper functions to define the curves

def window_fig(start, stop):
    ''' generates a figure with the raw data with window superimposed'''
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.update_layout(title='Raw Data and Window')
    fig.update_xaxes(title='Time (ps)')
    fig.update_yaxes(title='Amplitude (V)', secondary_y=False)
    fig.add_traces(data.window_curves(), secondary_ys=[True,True])
    fig.add_traces(data.raw_curves())
    fig.add_vline(x=stop/data.sample.sampling_rate)
    fig.add_vline(x=start/data.sample.sampling_rate)
    return fig

def processed_fig():
    '''generates the windowed and zero padded data plot'''
    fig = go.Figure()
    fig.update_layout(title='Windowed Signal (ZP)')
    fig.update_xaxes(title='Time (ps)')
    fig.update_yaxes(title='Amplitude (V)')
    fig.add_traces(data.processed_signal_curves())
    return fig
def spectral_fig():
    ''' generates the post fourier transformed spectrum plot'''
    fig = go.Figure()
    fig.update_layout(title='Amplitude of FFT (ZP)')
    fig.update_xaxes(title='Frequency (THz)')
    fig.update_yaxes(title='Amplitude (V)', type='log')
    fig.add_traces(data.spectral_curves())
    return fig
def phase_fig():
    '''generates the phase plot'''
    fig = go.Figure()
    fig.update_layout(title='Phase of FFT (ZP)')
    fig.update_xaxes(title='Frequency (THz)')
    fig.update_yaxes(title='Phase (radians)')
    fig.add_traces(data.unwrapped_phase_curves())
    fig.add_traces(data.wrapped_phase_curves())
    return fig

In [ ]:
## The dash app to display the figures and sliders

# read Bootstrap external theme
external_stylesheets = [dbc.themes.BOOTSTRAP]

app = Dash(__name__, external_stylesheets=external_stylesheets)

# app layout using bootstrap
app.layout = html.Div([
     dbc.Row([
                dbc.Col([
             dbc.Row(
                  dbc.Col(html.Div(html.H1('THz Spectral Processor', style={'textAlign': 'center'})))
             ),
             dbc.Row([
                dbc.Col(
                    html.Div([
                        html.Label("Window Start and End", htmlFor="window_range"),
                        dcc.RangeSlider(0, len(data.sample.amp),1,marks=None, value=[0,int(len(data.sample.amp)/10)],id='window_range')
                        # dcc.RangeSlider(0, 30000,1,marks=None, value=[0,(30000/10)],id='window_range')
                        ]), width = 6
                        ),
                dbc.Col(
                    html.Div([
                        html.Label("Window Curve", htmlFor="window_curve"),
                        dcc.Slider(0, 1,0.01, marks={(i/10): '{}'.format(i/10) for i in range(11)},
                        value=0.1,
                        id='window_curve')
                        ]) ,width = 6

             ),
             ]),
             dbc.Row([
                 dbc.Col(
                     html.Div([
                        html.Label("Zero Padding", htmlFor="zero_fill"),
                        dcc.Slider(2,20,1,marks={(i): '{}'.format(2 ** i) for i in range(2,21)},
                        value=0.1,
                        id='zero_fill')
                     ])
                 )
             ]),
             dbc.Row([
                  dbc.Col(html.Div(dcc.Graph(id='graph1')), width=6),
                  dbc.Col(html.Div(dcc.Graph(id='graph2')), width=6)
             ]),
             dbc.Row([
                  dbc.Col(html.Div(dcc.Graph(id='graph3')), width=6),
                  dbc.Col(html.Div(dcc.Graph(id='graph4')), width=6)
             ])


        ], width=12)

     
    ])
])

     
# callbacks from the slider updates
@callback(
    Output('graph1', 'figure',allow_duplicate=True),
    Output('graph2', 'figure',allow_duplicate=True),
    Output('graph3', 'figure',allow_duplicate=True),
    Output('graph4', 'figure',allow_duplicate=True),
    Input('window_range', 'value'),
    Input('window_curve', 'value'),
    Input('zero_fill', 'value'),
    prevent_initial_call=True
    )
def update_figure(rng, curve, zp):
    data.process_signals(rng[0],rng[1],curve,zp)
    fig=window_fig(rng[0],rng[1])
    fig2= processed_fig()
    fig3=spectral_fig()
    fig4=phase_fig()
    return fig,fig2,fig3,fig4

#callback for data selection     



if __name__ == '__main__':
    app.run(debug=True)